In [1]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher
from heterogt.utils.train import train_with_early_stopping
from heterogt.utils.seed import set_random_seed

In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-III",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"], 
    task_index = 0,
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]"],
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,   
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: death


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_gender_sentences = ["[PAD]"] + [str(c) + "_" + gender for c in set(ehr_full_data["AGE"].values.tolist()) for gender in ["M", "F"]]
token_type_sentences = ["[PAD]"] + config.token_type
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
task_sentences = config.tasks
tokenizer = EHRTokenizer(token_type_sentences, age_gender_sentences, task_sentences, diag_sentences, 
                         med_sentences, lab_sentences, pro_sentences, special_tokens=config.special_tokens)
config.label_vocab_size = len(tokenizer.diag_voc.id2word)  # only for diagnosis
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_gender_vocab_size = tokenizer.token_number("age_gender")
print(f"Age and gender vocabulary size: {config.age_gender_vocab_size}")

Age and gender vocabulary size: 37


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 28.648477157360407 %
Percentage of READMISSION in test dataset: 40.1491116751269 %
Percentage of STAY>7 days in test dataset: 50.58692893401015 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task)

In [10]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [11]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "prauc"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "prauc"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [12]:
input_ids, token_types, adm_index, age_gender_ids, task_index, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age/Sex IDs shape:", age_gender_ids.shape)
print("Task Index:", task_index)
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 256])
Token Types shape: torch.Size([32, 256])
Admission Index shape: torch.Size([32, 256])
Age/Sex IDs shape: torch.Size([32, 7])
Task Index: 0
Labels shape: torch.Size([32, 1])


# Model Walkthrough

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData, Batch as HeteroBatch
from torch_geometric.nn import HeteroConv, GATConv

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [14]:
class DiseaseOccHetGNN(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()
        self.conv1 = HeteroConv({
            ('visit','contains','occ'): GATConv(d_model, d_model, add_self_loops=False),
            ('occ','contained_by','visit'): GATConv(d_model, d_model, add_self_loops=False),
            ('visit','next','visit'): GATConv(d_model, d_model, add_self_loops=False),
        }, aggr='mean')
        self.lin = nn.Linear(d_model, d_model)
    
    def forward(self, hg: HeteroData):
        # x_dict: {'visit': [N_visit, d], 'occ': [N_occ, d]}
        x_dict = {'visit': hg['visit'].x, 'occ': hg['occ'].x}
        x_dict = self.conv1(x_dict, hg.edge_index_dict)
        x_dict = {k: self.lin(v) for k, v in x_dict.items()}
        return x_dict # {'visit': [N_visit, d], 'occ': [N_occ, d]}

In [15]:
# multi-class classification task
class MultiPredictionHead(nn.Module):
    def __init__(self, hidden_size, label_size):
        super(MultiPredictionHead, self).__init__()
        self.cls = nn.Sequential(
                nn.Linear(hidden_size, hidden_size), 
                nn.ReLU(), 
                nn.Linear(hidden_size, label_size)
            )

    def forward(self, input):
        return self.cls(input)
    
class BinaryPredictionHead(nn.Module):
    def __init__(self, hidden_size):
        super(BinaryPredictionHead, self).__init__()
        self.cls = nn.Sequential(
                nn.Linear(hidden_size, hidden_size), 
                nn.ReLU(), 
                nn.Linear(hidden_size, 1)
            )

    def forward(self, input):
        return self.cls(input)

In [16]:
for i in range(len(train_dataset)):
    age_gender_ids = train_dataset[i][3]
    if len(age_gender_ids[0]) > 3:
        print(age_gender_ids)
        break
exp_i = i
id_seq = torch.concat([train_dataset[exp_i][0][0], torch.zeros(5, dtype=train_dataset[exp_i][0][0].dtype)], dim=0)
type_seq = torch.concat([train_dataset[exp_i][1][0], torch.zeros(5, dtype=train_dataset[exp_i][1][0].dtype)], dim=0)
visit_seq = torch.concat([train_dataset[exp_i][2][0], torch.zeros(5, dtype=train_dataset[exp_i][2][0].dtype)], dim=0)
age_sex = torch.concat([train_dataset[exp_i][3][0], torch.zeros(3, dtype=train_dataset[exp_i][3][0].dtype)], dim=0)

tensor([[25, 25, 25, 11]])


In [ ]:
class HeteroGT(nn.Module):
    def __init__(self, tokenizer, d_model, num_heads, num_layers, max_num_adms, device, task, use_hetero_graph):
        super(HeteroGT, self).__init__()
        self.device = device
        self.tokenizer = tokenizer
        self.max_num_adms = max_num_adms
        self.use_hetero_graph = use_hetero_graph
        self.global_vocab_size = len(self.tokenizer.vocab.word2id)
        self.age_sex_vocab_size = len(self.tokenizer.age_gender_voc.word2id)
        self.n_type = len(self.tokenizer.token_type_voc.word2id)
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.seq_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="all")[0] #0
        self.type_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="type")[0] #0
        self.adm_pad_id = 0
        self.age_sex_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="age_gender")[0] #0
        self.diag_type_id = 1
        self.visit_type_id = 5
        
        # embedding layers
        self.token_emb = nn.Embedding(self.global_vocab_size, d_model, padding_idx=self.seq_pad_id)
        self.type_emb = nn.Embedding(self.n_type + 1, d_model, padding_idx=self.type_pad_id)
        self.adm_index_emb = nn.Embedding(self.max_num_adms + 1, d_model, padding_idx=self.adm_pad_id) # +1 for pad
        self.age_sex_emb = nn.Embedding(self.age_sex_vocab_size, d_model, padding_idx=self.age_sex_pad_id)
        self.task_emb = nn.Embedding(5, d_model, padding_idx=None)  # task embedding, not used in this model
        
        # GNN
        self.het_gnn = DiseaseOccHetGNN(d_model)    

        # encoder transformer
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, batch_first=True, norm_first = True)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers, enable_nested_tensor=False)

        # prediction head
        if task in ["death", "stay", "readmission"]:
            self.cls_head = BinaryPredictionHead(self.d_model)
        else:
            self.cls_head = MultiPredictionHead(self.d_model, config.label_vocab_size)


    def forward(self, input_ids, token_types, adm_index, age_gender_index, task_id):
        B, L = input_ids.shape
        task_id = torch.full((B,), task_id, dtype=torch.long, device=self.device)
        # 基础表示
        token_embed = self.token_emb(input_ids)  # [B, L, d]
        adm_emb  = self.adm_index_emb(adm_index)          # [B, L, d]
        type_emb = self.type_emb(token_types)       # [B, L, d]
        x_tokens = token_embed + adm_emb + type_emb  # [B, L, d]
        task_emb = self.task_emb(task_id).unsqueeze(1)           # [B, 1, d]
        x = torch.cat([task_emb, x_tokens], dim=1)  # [B, 1+L, d]
        
        # mask      
        seq_pad_mask = (input_ids == self.seq_pad_id)         # [B, L]
        task_pad_mask = torch.zeros((B, 1), dtype=torch.bool, device=self.device)
        mask = torch.concat([task_pad_mask, seq_pad_mask], dim=1)  # [B, 1+L]
        
        if self.use_hetero_graph:
            # get visit embed and mask
            visit_emb_pad, visit_pad_mask = self.visit_segment(B, input_ids, token_types, adm_index, age_gender_index)
            x = torch.cat([x, visit_emb_pad], dim=1)  # [B, 1+L(+V), d]
            mask = torch.concat([mask, visit_pad_mask], dim=1)

        
        # ===== Transformer 编码（batch_first=True） =====
        h = self.encoder(x, src_key_padding_mask=mask)   # [B, 1+L(+V), d]

        # ===== 分类：取 CLS（task 位） =====
        logits = self.cls_head(h[:, 0, :])  # [B, label_size]
        return logits

    def visit_segment(self, B, input_ids, token_types, adm_index, age_gender_index):
        graphs = []
        for p in range(B):
            hg_p = self.build_patient_graph(input_ids[p], token_types[p], adm_index[p], age_gender_index[p])
            graphs.append(hg_p)

        batch_graph = HeteroBatch.from_data_list(graphs).to(self.device)
        out = self.het_gnn(batch_graph)
        h_visit_all = out['visit']  # extract virtual visit node representations

        # 取出每个样本的 visit 表示序列（按我们在 build_patient_graph 中的保序构造）
        visit_emb_seq = []
        offset = 0
        for p in range(B):
            n_v = graphs[p]['visit'].num_nodes
            visit_emb_p = h_visit_all[offset:offset + n_v]  # [N_visit_p, d]
            offset += n_v
            visit_emb_seq.append(visit_emb_p)
            
        visit_emb_pad = []
        visit_pad_mask = []
        visit_index_pad = []
        for p in range(B):
            v = visit_emb_seq[p]                          # [N_visit_p, d]
            Np = v.size(0)
            if Np < self.max_num_adms:
                pad_len = self.max_num_adms - Np
                v_pad = torch.cat([v, torch.zeros(pad_len, self.d_model, device=self.device, dtype=v.dtype)], dim=0)
                m_pad = torch.cat([torch.zeros(Np, dtype=torch.bool, device=self.device), torch.ones(pad_len, dtype=torch.bool, device=self.device)], dim=0)
                i_pad = torch.cat([torch.arange(1, Np + 1, device=self.device), torch.full((pad_len,), self.adm_pad_id, dtype=torch.long, device=self.device)], dim=0)
            else:
                v_pad = v[:self.max_num_adms]
                m_pad = torch.zeros(self.max_num_adms, dtype=torch.bool, device=self.device)
                i_pad = torch.arange(1, self.max_num_adms + 1, device=self.device)
            visit_emb_pad.append(v_pad)      # [V_max, d]
            visit_pad_mask.append(m_pad)     # [V_max]
            visit_index_pad.append(i_pad)    # [V_max]

        visit_emb_pad  = torch.stack(visit_emb_pad,  dim=0)  # [B, V_max, d]
        visit_pad_mask = torch.stack(visit_pad_mask, dim=0)  # [B, V_max]
        visit_index_pad = torch.stack(visit_index_pad, dim=0)  # [B, V_max]

        # ====== 对齐与类型嵌入（关键部分） ======
        nonpad = (~visit_pad_mask).unsqueeze(-1)             # [B, V_max, 1], bool

        # 1. 加 type embedding（仅非 pad 位）
        visit_type_ids = torch.full((B, self.max_num_adms), self.visit_type_id, dtype=torch.long, device=self.device) # [B, V_max]                                                   # [B, V_max]
        visit_type_emb = self.type_emb(visit_type_ids) * nonpad      # [B, V_max, d]
        
        # 2. 加 visit index embedding（仅非 pad 位）
        visit_index_emb = self.adm_index_emb(visit_index_pad) * nonpad  # [B, V_max, d]
        
        # 3. 得到最终embedding
        visit_emb_pad  = visit_emb_pad + visit_type_emb + visit_index_emb

        return visit_emb_pad, visit_pad_mask
    
    def build_patient_graph(self, id_seq: torch.Tensor, type_seq: torch.Tensor, visit_seq: torch.Tensor, age_sex: torch.Tensor):
        # build a graph just for one patient
        hg = HeteroData()
        occ_mask = (type_seq == self.diag_type_id) & (id_seq != self.seq_pad_id) # 疾病token mask
        occ_pos = torch.nonzero(occ_mask, as_tuple=False).view(-1) # 疾病 token 的位置索引，形状 [N_occ]
        N_occ = occ_pos.numel() # 疾病 token 数量

        # build visit virtual nodes
        nonpad = id_seq != self.seq_pad_id
        visit_used = visit_seq[nonpad] # seq非pad部分
        visit_ids_unique, visit_lid_nonpad = torch.unique(visit_used, return_inverse=True)
        visit_lid_full = torch.full_like(id_seq, fill_value=-1)
        visit_lid_full[nonpad] = visit_lid_nonpad
        N_visit = visit_ids_unique.numel()
        age_sex_nonpad = age_sex[age_sex!=self.age_sex_pad_id]
        assert N_visit == len(visit_ids_unique) == len(age_sex_nonpad)
        visit_x = self.age_sex_emb(age_sex_nonpad.to(self.device))
        hg['visit'].x = visit_x
        hg['visit'].num_nodes = N_visit
        
        # build diag nodes
        gid_occ = id_seq[occ_pos]
        x_occ = self.token_emb(gid_occ) # [N_occ, d]
        hg['occ'].x = x_occ
        hg['occ'].num_nodes = N_occ

        # build edges between diag nodes and virtual visit nodes
        occ_visit_lid = visit_lid_full[occ_pos]
        e_v2o = torch.stack([occ_visit_lid, torch.arange(N_occ, device=self.device)], dim=0)
        e_o2v = torch.stack([torch.arange(N_occ, device=self.device), occ_visit_lid], dim=0)
        hg['visit','contains','occ'].edge_index = e_v2o
        hg['occ','contained_by','visit'].edge_index = e_o2v
        
        # build forward edges between virtual visit nodes
        if N_visit > 1:
            src = torch.arange(0, N_visit - 1, device=self.device)
            dst = torch.arange(1, N_visit, device=self.device)
            e_next = torch.stack([src, dst], dim=0) # [2, N_visit-1]
        else:
            e_next = torch.empty(2, 0, dtype=torch.long, device=self.device)
        hg['visit','next','visit'].edge_index = e_next
        return hg

In [31]:
final_metrics = {"precision":[],"recall":[],"f1":[],"auc":[],"prauc":[]}
for i in range(5):
    model = HeteroGT(tokenizer, d_model=128, num_heads=4, num_layers=2, max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, use_hetero_graph=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    for key in final_metrics.keys():
        final_metrics[key].append(best_test_metric[key])

Epoch 001:   0%|          | 0/98 [00:00<?, ?it/s, loss=0.7180]

Running inference: 100%|██████████| 197/197 [00:02<00:00, 68.89it/s]


Validation: {'precision': 0.7248868778214943, 'recall': 0.4720094284002828, 'f1': 0.5717344705938429, 'auc': 0.8336861788333322, 'prauc': 0.6783495571486897}
Test:      {'precision': 0.7383773928829683, 'recall': 0.4485049833862209, 'f1': 0.5580433986702152, 'auc': 0.8372240744871475, 'prauc': 0.696012729040115}


Running inference: 100%|██████████| 197/197 [00:02<00:00, 69.10it/s]


Validation: {'precision': 0.9137055837099641, 'recall': 0.10606953447197368, 'f1': 0.19007391576874788, 'auc': 0.8554749260404066, 'prauc': 0.7136796609242874}
Test:      {'precision': 0.9317073170277216, 'recall': 0.10575858250218297, 'f1': 0.18995524431335267, 'auc': 0.8573256626422375, 'prauc': 0.7273704931597026}


...

Validation: {'precision': 0.6533110740086071, 'recall': 0.6918090748338727, 'f1': 0.6720091535577762, 'auc': 0.8686758783336426, 'prauc': 0.7534936181795037}
Test:      {'precision': 0.6856840993096373, 'recall': 0.7187153931300182, 'f1': 0.7018112953504178, 'auc': 0.8699162836983781, 'prauc': 0.7651372878554726}


Running inference: 100%|██████████| 197/197 [00:02<00:00, 69.44it/s]


Validation: {'precision': 0.6695652173874228, 'recall': 0.6806128461951644, 'f1': 0.6750438290115853, 'auc': 0.8701919877224329, 'prauc': 0.7579580581757852}
Test:      {'precision': 0.6937984496085615, 'recall': 0.6937984496085615, 'f1': 0.6937984446085615, 'auc': 0.8762996424644497, 'prauc': 0.7797466763174398}


Running inference: 100%|██████████| 197/197 [00:03<00:00, 62.42it/s]

Validation: {'precision': 0.5881548974916257, 'recall': 0.7607542722406556, 'f1': 0.6634121225193575, 'auc': 0.8622990470041814, 'prauc': 0.7505121227553713}
Test:      {'precision': 0.6036846615226921, 'recall': 0.7801771871496115, 'f1': 0.6806763235804547, 'auc': 0.8674740145367918, 'prauc': 0.7734520232562784}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.7322936972012197, 'recall': 0.6641131408328573, 'f1': 0.6965389319668238, 'auc': 0.8854773109889922, 'prauc': 0.7829647952819372}
Corresponding test performance:
{'precision': 0.7612219451324114, 'recall': 0.6760797342155256, 'f1': 0.7161290272714099, 'auc': 0.8870165994779518, 'prauc': 0.7949989873825286}


In [32]:
# print the mean and std of the final metrics
print("\nFinal Metrics:")
for key in final_metrics.keys():
    mean_value = np.mean(final_metrics[key])
    std_value = np.std(final_metrics[key])
    print(f"{key}: {mean_value:.4f} ± {std_value:.4f}")


Final Metrics:
precision: 0.7275 ± 0.0541
recall: 0.6977 ± 0.0451
f1: 0.7089 ± 0.0067
auc: 0.8896 ± 0.0061
prauc: 0.7977 ± 0.0102


In [33]:
final_metrics = {"precision":[],"recall":[],"f1":[],"auc":[],"prauc":[]}
for i in range(5):
    model = HeteroGT(tokenizer, d_model=128, num_heads=4, num_layers=2, max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, use_hetero_graph=False).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    for key in final_metrics.keys():
        final_metrics[key].append(best_test_metric[key])

Running inference: 100%|██████████| 197/197 [00:00<00:00, 328.78it/s]


Validation: {'precision': 0.825688073369245, 'recall': 0.1591043017079605, 'f1': 0.2667984162605112, 'auc': 0.8247873953104016, 'prauc': 0.6534444119908926}
Test:      {'precision': 0.815340909067746, 'recall': 0.1589147286812906, 'f1': 0.26598702229054805, 'auc': 0.830052867104218, 'prauc': 0.6700850619612017}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 328.70it/s]


Validation: {'precision': 0.6424452133757644, 'recall': 0.6564525633432148, 'f1': 0.6493733555330831, 'auc': 0.849634420722559, 'prauc': 0.698726053965898}
Test:      {'precision': 0.6787983824339758, 'recall': 0.6506090808380365, 'f1': 0.6644048578766366, 'auc': 0.8538049025849805, 'prauc': 0.7137733147772144}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 326.14it/s]


...

Validation: {'precision': 0.819023569016675, 'recall': 0.5733647613401688, 'f1': 0.6745233920313775, 'auc': 0.8976993102299475, 'prauc': 0.799144322498208}
Test:      {'precision': 0.8313253011981421, 'recall': 0.5730897009935045, 'f1': 0.6784660718607664, 'auc': 0.8929484840561599, 'prauc': 0.8056489701480083}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 323.73it/s]


Validation: {'precision': 0.6970198675458222, 'recall': 0.7442545668783486, 'f1': 0.7198632038926894, 'auc': 0.8952285188242958, 'prauc': 0.7889977236877954}
Test:      {'precision': 0.6935312831352412, 'recall': 0.7242524916903419, 'f1': 0.7085590415857249, 'auc': 0.8853061678206187, 'prauc': 0.7892736951342193}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 325.95it/s]

Validation: {'precision': 0.8038834951378264, 'recall': 0.4879198585710789, 'f1': 0.6072607213672778, 'auc': 0.8675190441401992, 'prauc': 0.7473270407589773}
Test:      {'precision': 0.8159111933319528, 'recall': 0.4883720930205517, 'f1': 0.6110148896650887, 'auc': 0.8655265512369962, 'prauc': 0.7617726055287253}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.8037313432775841, 'recall': 0.6346493812573091, 'f1': 0.7092525469248081, 'auc': 0.9125310206187555, 'prauc': 0.8206303116089178}
Corresponding test performance:
{'precision': 0.8016997167082033, 'recall': 0.6267995570286445, 'f1': 0.7035425680973054, 'auc': 0.9054422243526963, 'prauc': 0.8219136048179316}


In [35]:
# print the mean and std of the final metrics
print("\nFinal Metrics:")
for key in final_metrics.keys():
    mean_value = np.mean(final_metrics[key])
    std_value = np.std(final_metrics[key])
    print(f"{key}: {mean_value:.4f} ± {std_value:.4f}")


Final Metrics:
precision: 0.7206 ± 0.0609
recall: 0.6940 ± 0.0604
f1: 0.7023 ± 0.0140
auc: 0.8895 ± 0.0128
prauc: 0.7922 ± 0.0237
